In [11]:
import pandas as pd
import re
import spacy
nlp = spacy.load("en_core_web_sm")

pronoun_testcases = pd.read_csv("/content/pronoun_testcases.csv")
pronoun_testcases.head()

,input_text,target_gender,expected_output
0,He is going to the market.,female,She is going to the market.
1,His book is on the table.,female,Her book is on the table.
2,I saw him yesterday.,female,I saw her yesterday.
3,He hurt himself.,female,She hurt herself.
4,I called him last night.,female,I called her last night.


In [12]:
def transform_pronouns(text):
    doc = nlp(text)

    male_to_female = {
        'he': 'she', 'He': 'She', 'HE': 'SHE',
        'him': 'her', 'Him': 'Her', 'HIM': 'HER',
        'his': 'her', 'His': 'Her', 'HIS': 'HER',
        'himself': 'herself', 'Himself': 'Herself', 'HIMSELF': 'HERSELF'
    }

    female_to_male = {
        'she': 'he', 'She': 'He', 'SHE': 'HE',
        'herself': 'himself', 'Herself': 'Himself', 'HERSELF': 'HIMSELF'
    }

    result = ""

    for i, token in enumerate(doc):
        word = token.text
        transformed = word

        if word in male_to_female:
            transformed = male_to_female[word]
        elif word in female_to_male:
            transformed = female_to_male[word]
        elif word.lower() in ['her', 'Her', 'HER']:
            if token.tag_ == 'PRP$':
                transformed = {'her': 'his', 'Her': 'His', 'HER': 'HIS'}[word]
            else:
                transformed = {'her': 'him', 'Her': 'Him', 'HER': 'HIM'}[word]

        result += transformed

        if i < len(doc) - 1:
            result += token.whitespace_

    return result

In [13]:

pronoun_testcases['predicted_output'] = pronoun_testcases['input_text'].apply(transform_pronouns)

print(pronoun_testcases[['expected_output', 'predicted_output']].head())

pronoun_accuracy = (pronoun_testcases['expected_output'] == pronoun_testcases['predicted_output']).mean() * 100
print(f"\nPronoun Transformation Accuracy: {pronoun_accuracy:.2f}%")

               expected_output             predicted_output
0  She is going to the market.  She is going to the market.
1    Her book is on the table.    Her book is on the table.
2         I saw her yesterday.         I saw her yesterday.
3            She hurt herself.            She hurt herself.
4     I called her last night.     I called her last night.

Pronoun Transformation Accuracy: 100.00%
